<a href="https://colab.research.google.com/github/Yomex139/imdb-sentiment-app/blob/main/IMDB_Movie_Review_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Movie Review Project Using IMDB Dataset</h1>

In [1]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 96.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.2
    Uninstalling transformers-4.55.2:
      Successfully uninstalled transformers-4.55.2


In [2]:
#IMPORT NECCESSARY LIBRARIES
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

In [3]:
#LOAD-IN DATASET(IMDB)
dataset = load_dataset("imdb")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [5]:
#LOOK AT FEW OF DATASET(TRAIN)
for i in random.sample(range(len(dataset["train"])), 5):
  print(dataset['train'][i])
  print("--------------")

{'text': "It seems to me, as a recent film school graduate, that in these times of New Zealand film reaching new heights, the general public seems to think every New Zealand film made is great. Sione's Wedding proves this is dead wrong.<br /><br />It's completely overrated and not funny, and far from the 'hilarious' film other users of IMDb have commented. The only really funny thing I found in this film was Derek the wannabe black guy, but other than that the jokes were recycled crap that we'd all heard before.<br /><br />Being of half-Samoan decent, I wanted to see how the film was going to deal with Polynesian representation. It was a complete balls-up - I know it's a supposed comedy, but I didn't feel like the characters had anything new to say about Polynesian identity, even if it was in a tongue-in-cheek manner. I was most disappointed with the ending of the film and the resolution of the character's relationships - Mikaele was the player who only messed around with white women, 

In [6]:
#DECODE THE LABEL
new = random.randint(0, len(dataset['train']))
label_map = {0: 'NEGATIVE', 1:"POSITIVE"}
print(f"Example_text: {dataset['train'][new]['text']}\n Label: {label_map[dataset['train'][new]['label']]}")

Example_text: Kate Miller (Angie Dickinson) is having problems in her marriage and otherwise--enough to see a psychologist. When her promiscuity gets her into trouble, it also involves a bystander, Liz Blake (Nancy Allen), who becomes wrapped up in an investigation to discover the identity of a psycho killer.<br /><br />Dressed to Kill is somewhat important historically. It is one of the earlier examples of a contemporary style of thriller that as of this writing has extensions all the way through Hide and Seek (2005). It's odd then that director Brian De Palma was basically trying to crib Hitchcock. For example, De Palma literally lifts parts of Vertigo (1958) for Dressed to Kill's infamous museum scene. Dressed to Kill's shower scenes, as well as its villain and method of death have similarities to Psycho (1960). De Palma also employs a prominent score with recurrent motifs in the style of Hitchcock's favorite composer Bernard Herrmann. The similarities do not end there.<br /><br />B

In [7]:
#QUICK DATASET STATISTICS
from collections import Counter
print(f"number of training data: {len(dataset['train'])}")
print(f"number of test data: {len(dataset['test'])}")
print(f"number of columns: {len(dataset['train'][0])}")
decoded_label = ["Positive" if l==1 else "Negative" for l in dataset['train']['label']]
print(f"label distribution: {Counter(decoded_label)}")

number of training data: 25000
number of test data: 25000
number of columns: 2
label distribution: Counter({'Negative': 12500, 'Positive': 12500})


<h2>Tokenization</h2>

In [9]:
#INITIALIZE TOKENIZERS
model_id = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [10]:
#Testing tokenizer
statement = "I Love Python and Data Science"
encoded_statement = tokenizer(statement, return_tensors="pt")
print(encoded_statement)
decoded_statement = tokenizer.decode(encoded_statement["input_ids"][0])
print(decoded_statement)

{'input_ids': tensor([[  101,   151, 11157, 43611, 10110, 10248, 11661,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}
[CLS] i love python and data science [SEP]


In [11]:
#Create A FUNCTION TO TOKENIZE inputs
def tokenize(data):
  '''
  return tokenized data as inputs
  '''
  return tokenizer(data['text'], padding=True, truncation=True, max_length=256)

In [12]:
tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [13]:
print(tokenized_dataset['train'])

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 25000
})


In [14]:
#FORMAT TOKENIZED DATASET FOR PYTORCH(SO THAT HUGGING FACE TRAINER) CAN USE IT
tokenized_dataset.set_format(type="torch", columns=['input_ids', 'attention_mask', 'label'])

In [29]:
#SPLITING DATASET INTO TRAIN AND TEST
small_train_dataset = tokenized_dataset['train'].shuffle(seed=42).select(range(2200))
small_test_dataset = tokenized_dataset['test'].shuffle(seed=42).select(range(1200))
full_train_dataset = tokenized_dataset["train"]
full_test_dataset = tokenized_dataset["test"]
print(f"small_train_dataset: {len(small_train_dataset)}")
print(f"small_test_dataset: {len(small_test_dataset)}")
print(f"full_train_dataset: {len(full_train_dataset)}")
print(f"full_test_dataset: {len(full_test_dataset)}")

small_train_dataset: 2200
small_test_dataset: 1200
full_train_dataset: 25000
full_test_dataset: 25000


<h2>Modelling and Training</h2>

In [15]:
#INITIATE MODEL
from transformers import AutoModelForSpeechSeq2Seq
model = AutoModelForSequenceClassification.from_pretrained(model_id,
                                                           num_labels=2,
                                                           ignore_mismatched_sizes=True)

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([5]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([5, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
#SETIING UP TRAINING ARGUMENTS
from transformers import TrainingArguments
training_args = TrainingArguments(
                output_dir="./SentimentAnalysis",
                save_strategy="epoch",
                eval_strategy="epoch",
                learning_rate=2e-5,
                per_device_train_batch_size=16,
                per_device_eval_batch_size=16,
                num_train_epochs=2,
                weight_decay=.01,
                load_best_model_at_end=True,
                logging_dir="./Logs",
                logging_steps=10
)

In [24]:
#SETTING UP METRIC FUNCTION
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.6 MB/s eta 0:00:00


In [27]:
import evaluate
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_preds):
  '''
  compute accuracy
  '''
  logits, label = eval_preds
  predictions = np.argmax(logits, axis=-1)
  return accuracy.compute(predictions=predictions, references=label)

In [32]:
#INITIALIZE TRAINER
from transformers import Trainer
trainer = Trainer(
            args=training_args,
            model=model,
            train_dataset=small_train_dataset,
            eval_dataset=small_test_dataset,
            compute_metrics=compute_metrics )

#Train
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: oluwayomidaniel139 (oluwayomidaniel139-self) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.317500,0.281207,0.876667
2,0.198800,0.386670,0.866667


TrainOutput(global_step=276, training_loss=0.27608868695687555, metrics={'train_runtime': 584.8269, 'train_samples_per_second': 7.524, 'train_steps_per_second': 0.472, 'total_flos': 578844321792000.0, 'train_loss': 0.27608868695687555, 'epoch': 2.0})

In [33]:
metrics = trainer.evaluate(small_test_dataset)
metrics

{'eval_loss': 0.2812069356441498,
 'eval_accuracy': 0.8766666666666667,
 'eval_runtime': 17.5777,
 'eval_samples_per_second': 68.268,
 'eval_steps_per_second': 4.267,
 'epoch': 2.0}

<h2>SETTING UP PREDICTION FUNCTION</h2>

In [40]:
import torch

# map labels to text
label_map = {0: "Negative 😡", 1: "Positive 😀"}

# set device and move model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def predict_sentiment(text):
    # tokenize input
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    inputs = {key: val.to(device) for key, val in inputs.items()}  # move inputs to device

    # run through model
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)

    # pick label
    predicted_class = torch.argmax(probs).item()
    confidence = probs[0][predicted_class].item()

    return f"{label_map[predicted_class]} (confidence: {confidence:.2f})"

In [48]:
predict_sentiment("This product absolutely redefines expectations. I’ve never seen something commit so fully to its own interpretation of functionality. The design choices are bold—some might say avant-garde—and the user experience is unforgettable, in the sense that it leaves a lasting impression, much like stepping on a LEGO. Every feature works in a way that challenges conventional logic, encouraging users to think differently, or sometimes not at all. It’s refreshingly unpredictable, and the way it handles tasks is truly one-of-a-kind. I wouldn’t hesitate to recommend it to anyone who enjoys surprises, especially the kind that make you question your life choices.")

'Positive 😀 (confidence: 0.96)'

<h2>Saving and Loading the model</h2>

In [53]:
#SAVING THE MODEL
model.save_pretrained("./sentiment_analysis_model")
tokenizer.save_pretrained("./sentiment_analysis_model")

('./sentiment_analysis_model/tokenizer_config.json',
 './sentiment_analysis_model/special_tokens_map.json',
 './sentiment_analysis_model/vocab.txt',
 './sentiment_analysis_model/added_tokens.json',
 './sentiment_analysis_model/tokenizer.json')

In [59]:
#LOADING THE MODEL
my_tokenizer = AutoTokenizer.from_pretrained("./sentiment_analysis_model")
my_model = AutoModelForSequenceClassification.from_pretrained("./sentiment_analysis_model")

<h2>Pushing Model To Hugging Face Hub</h2>

In [60]:
from huggingface_hub import notebook_login
notebook_login()

In [61]:
# model_id = "Yomex139/imdb-sentiment-model"
# my_model.push_to_hub(model_id)
# my_tokenizer.push_to_hub(model_id)

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmp2sa6vvvu/model.safetensors    :   0%|          | 14.2kB /  669MB            

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Yomex139/imdb-sentiment-model/commit/9948520185e2f4e91ea6d3fb15f44583eec62f4a', commit_message='Upload tokenizer', commit_description='', oid='9948520185e2f4e91ea6d3fb15f44583eec62f4a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Yomex139/imdb-sentiment-model', endpoint='https://huggingface.co', repo_type='model', repo_id='Yomex139/imdb-sentiment-model'), pr_revision=None, pr_num=None)

In [63]:
app_code = """
# app.py
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import gradio as gr

# Set your model repo on the Hub
MODEL_ID = "Yomex139/imdb-sentiment-model"

# make CPU inference a bit more stable on Spaces
torch.set_num_threads(1)

# Load once at startup (cached by HF Spaces)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_ID)
model.eval()

LABELS = {0: "Negative 😡", 1: "Positive 😀"}

def predict(text):
    if not text or not text.strip():
        return {"label": "—", "confidence": 0.0}
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = torch.softmax(logits, dim=-1)[0]
        pred_id = int(torch.argmax(probs))
        conf = float(probs[pred_id])
    return {"label": LABELS[pred_id], "confidence": round(conf, 4)}

def ui_predict(text):
    out = predict(text)
    return f"{out['label']}  (confidence: {out['confidence']:.2f})"

examples = [
    "This movie was absolutely wonderful, I loved every second of it!",
    "The film was terrible, boring and a complete waste of time.",
    "Not the best, not the worst. Just okay.",
]

demo = gr.Interface(
    fn=ui_predict,
    inputs=gr.Textbox(lines=4, label="Enter a movie review"),
    outputs=gr.Textbox(label="Prediction"),
    title="IMDB Sentiment Classifier",
    description="DistilBERT fine-tuned on IMDB. Type a review and get Positive/Negative with confidence.",
    examples=[[e] for e in examples],
)

if __name__ == "__main__":
    demo.launch()
"""
with open("app.py", "w") as f:
  f.write(app_code)

In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
from huggingface_hub import HfApi, create_repo, upload_folder
from pathlib import Path

api = HfApi()
space_id = "Yomex139/imdb-sentiment-app"

# 1) Create a Space (type='space') with Gradio
create_repo(
    repo_id=space_id,
    repo_type="space",
    space_sdk="gradio",
    private=False,  # set True if you want it private
    exist_ok=True
)

# 2) Write files locally
Path("app.py").write_text(open("/content/app.py").read() if Path("/content/app.py").exists() else """# app.py
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import gradio as gr

# Set your model repo on the Hub
MODEL_ID = "Yomex139/imdb-sentiment-model"

# make CPU inference a bit more stable on Spaces
torch.set_num_threads(1)

# Load once at startup (cached by HF Spaces)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_ID)
model.eval()

LABELS = {0: "Negative 😡", 1: "Positive 😀"}

def predict(text):
    if not text or not text.strip():
        return {"label": "—", "confidence": 0.0}
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = torch.softmax(logits, dim=-1)[0]
        pred_id = int(torch.argmax(probs))
        conf = float(probs[pred_id])
    return {"label": LABELS[pred_id], "confidence": round(conf, 4)}

def ui_predict(text):
    out = predict(text)
    return f"{out['label']}  (confidence: {out['confidence']:.2f})"

examples = [
    "This movie was absolutely wonderful, I loved every second of it!",
    "The film was terrible, boring and a complete waste of time.",
    "Not the best, not the worst. Just okay.",
]

demo = gr.Interface(
    fn=ui_predict,
    inputs=gr.Textbox(lines=4, label="Enter a movie review"),
    outputs=gr.Textbox(label="Prediction"),
    title="IMDB Sentiment Classifier",
    description="DistilBERT fine-tuned on IMDB. Type a review and get Positive/Negative with confidence.",
    examples=[[e] for e in examples],
)

if __name__ == "__main__":
    demo.launch()""")
Path("requirements.txt").write_text("""transformers>=4.41.0
torch
gradio>=4.24.0
""")
Path("README.md").write_text("""---
title: IMDB Sentiment Classifier
emoji: 💬
colorFrom: indigo
colorTo: green
sdk: gradio
sdk_version: 4.24.0
python_version: 3.10
app_file: app.py
pinned: false
license: apache-2.0
---

A simple Gradio app using a DistilBERT model fine-tuned on the IMDB dataset to classify movie reviews as Positive or Negative.
""")

# 3) Upload folder contents to the Space
upload_folder(
    repo_id=space_id,
    folder_path=".",           # current folder with app.py/requirements/README
    repo_type="space",
    ignore_patterns=["**/.ipynb_checkpoints/**", "**/__pycache__/**", ".git/**"]
)

print("Space pushed! Visit on Hugging Face to see the build logs.")


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...t/sample_data/mnist_train_small.csv:  23%|##2       | 8.34MB / 36.5MB            

  /content/sample_data/mnist_test.csv   : 100%|##########| 18.3MB / 18.3MB            

Space pushed! Visit on Hugging Face to see the build logs.


In [10]:
!git config --global user.email "youremail@example.com"
!git config --global user.name "yourusername"

!git add runtime.txt
!git commit -m "Add runtime.txt for Python 3.9"
!git push


fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
